## 使用PYSR库寻找数据间的显式形式

In [ ]:
## 导入数据
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. 数据读取和预处理
# 加载数据
df = pd.read_csv('../improved_data/xy_orbit.csv')
array = df.to_numpy()
array = array.T
t = array[0]
yy = array[1:]

# 提取坐标和速度，注意单位一致
x , y, vx, vy = yy[0]/1e10, yy[1]/1e10,  yy[2]/1e10,  yy[3]/1e10

# 计算距离 r 和加速度分量
r = np.sqrt(x**2 + y**2)
G = 6.67430e-11 * 86400**2/1e30  # 引力常数, m^3 kg^-1 s^-2（调整单位）
M = 1.989e30  # 太阳质量
GM = G * M
print(GM)
# 加速度分量
x_ddot = -GM * x / r**3
y_ddot = -GM * y / r**3

'''plt.figure(figsize=(8, 6))
plt.plot(x,y, color = 'red')
plt.xlabel('x(1e10 m)')
plt.ylabel('y(1e10 m)')
plt.title('Trajectory of the System')
plt.legend()
plt.grid()
plt.axis('equal')  # 确保 x 和 y 轴比例相同
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(t,vx, color = 'red',label = 'vx')
plt.plot(t,vy, color = 'green',label = 'vy')
plt.xlabel('t')
plt.ylabel('vx or vy (1e10 m/day)')
plt.title('speed of the System')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(t,x_ddot, color = 'red',label = 'x_ddot')
plt.plot(t,y_ddot, color = 'green',label = 'y_ddot')
plt.xlabel('t')
plt.ylabel('ax or ay (1e10 m/day^2)')
plt.title('acceleration of the System')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(t,x, color = 'blue',label = 'x')
plt.plot(t,y, color = 'black',label = 'y')
plt.xlabel('t')
plt.ylabel('x or y (1e10 m)')
plt.title('position of the System')
plt.legend()
plt.grid()
plt.show()'''


print(x.shape,y.shape, r.shape)

In [ ]:
from pysr import PySRRegressor

# 目标输入
new_colum = np.full(x.shape,GM)
X = np.stack([x,y,r,new_colum],axis=0).T

# 目标输出
Y = np.stack([x_ddot,y_ddot],axis=0).T

print(X[:5],Y[:5])


# 定义 PySR 模型
model = PySRRegressor(
    niterations=100,  # 迭代次数
    populations=30,  # 种群数量
    binary_operators=["+", "-", "*", "/"],  # 二元操作符
    unary_operators=["sin", "cos", "square"],  # 一元操作符
    elementwise_loss="L2DistLoss()",  # 损失函数
    parallelism='multithreading',  # 启用多线程
    verbosity=1,  # 显示详细日志 (verbosity 而不是 verbose)
)

# 拟合模型
print("Fitting model...")
model.fit(X, Y)

# 输出最佳表达式
for i, expr in enumerate(model.equations_):
    print(f"Best Expression for output {i+1}: {expr}")

# 预测
y_pred = model.predict(X)

# 评估
mse = np.mean((Y - y_pred) ** 2)
print(f"Mean Squared Error: {mse}")


In [ ]:
#将形式转化为符号表达式

import sympy as sp

# 将 PySR 表达式转换为 sympy 表达式
sympy_expr1 = model.sympy()[0]  # 第一个输出变量的表达式
sympy_expr2 = model.sympy()[1]  # 第二个输出变量的表达式

# 打印 sympy 表达式
print(f"SymPy Expression for output 1: {sympy_expr1}")
print(f"SymPy Expression for output 2: {sympy_expr2}")
